In [1]:
import numpy as np
from pymatgen.core import Structure, PeriodicSite, DummySpecie
# from pymatgen.analysis.graphs import StructureGraph
from pymatgen.analysis.local_env import MinimumDistanceNN

In [2]:
# Load defective structure
from pathlib import Path
defective_file_path = Path(r'dataset\high_MoS2\cifs\MoS2_Mo56W4Se1S122_dc05c576-af56-4203-89ea-12c432a1a34b.cif')
defective_struct = Structure.from_file(defective_file_path)
struct_lattice = defective_struct.lattice

# Get reference structure
ref_file_path = Path(r'dataset\high_MoS2\MoS2.cif')
ref_unit_cell = Structure.from_file(ref_file_path)
reference_struct = ref_unit_cell.make_supercell([8,8,1])


c:\Users\123\anaconda3\envs\msproject\Lib\site-packages\pymatgen\core\structure.py:3107: UserWarning: Issues encountered while parsing CIF: 45 fractional coordinates rounded to ideal values to avoid issues with finite precision.
  struct = parser.parse_structures(primitive=primitive)[0]
c:\Users\123\anaconda3\envs\msproject\Lib\site-packages\pymatgen\core\structure.py:3107: UserWarning: Issues encountered while parsing CIF: 6 fractional coordinates rounded to ideal values to avoid issues with finite precision.
  struct = parser.parse_structures(primitive=primitive)[0]


In [7]:
def struct_to_dict(structure):
    list_of_sites = structure.sites
    list_of_frac_coords = np.round(structure.frac_coords,3)
    structure_dict = {i: j for i, j in zip(list_of_sites, list_of_frac_coords)}
    return structure_dict

In [8]:
def get_defects(defective_struct, reference_struct):
    copy_defective_struct = defective_struct.copy()
    # struct to dict
    defective_dict = struct_to_dict(copy_defective_struct)
    reference_dict = struct_to_dict(reference_struct)

    # Get lattice of defective structure
    structure_lattice = copy_defective_struct.lattice

    # Define list to add all defect sites
    defects_list = []

    # Define a dictionary to hold properties
    sites_properties = {} # defect + nondefect sites
    defects_properties = {} # defect sites only

    for ref_site, ref_coords in reference_dict.items():
        matching = False
        for def_site, def_coords in defective_dict.items():
            if np.array_equal(ref_coords, def_coords):
                matching = True
                if ref_site.specie != def_site.specie: # Substitution case
                    # Add site to defects list
                    defects_list.append(def_site)

                    # Get atomic number change and defect type
                    add_property = {"original_an":ref_site.specie.Z,
                                    "new_an": def_site.specie.Z,
                                    "an_change": def_site.specie.Z - ref_site.specie.Z,
                                    "vacancy_defect": 0.0,
                                    "substitution_defect": 1.0}
                    defects_properties[def_site] = add_property
                    sites_properties[def_site] = add_property
                else:
                    add_property = {"original_an":ref_site.specie.Z, 
                                    "new_an": def_site.specie.Z,
                                    "an_change": def_site.specie.Z - ref_site.specie.Z,
                                    "vacancy_defect": 0.0,
                                    "substitution_defect": 0.0}
                    sites_properties[def_site] = add_property

                    

        if not matching: # Vacancy case
            # Add site to defective structure
            vacant_site = PeriodicSite(
                species= DummySpecie(),
                coords= ref_coords,
                coords_are_cartesian= False, 
                lattice= structure_lattice
                )
            
            copy_defective_struct.append(vacant_site.species, vacant_site.frac_coords)

            # Add site to defects list
            defects_list.append(vacant_site)

            # Get atomic number change and defect type
            add_property={"original_an":ref_site.specie.Z,
                          "new_an": 0,
                          "an_change": 0 - ref_site.specie.Z,
                          "vacancy_defect": 1.0,
                          "substitution_defect": 0.0}
            defects_properties[vacant_site] = add_property
            sites_properties[vacant_site] = add_property

    return copy_defective_struct, sites_properties, defects_list, defects_properties


# Get defect structure
new_defective_struct, new_defective_struct_properties, defect_sites, defect_properties = get_defects(defective_struct, reference_struct)


In [9]:
def prop_in_struct(properties, structure):
    # Create copy of structure
    copy_structure = structure.copy()

    # Add properties to structure
    for a_site in copy_structure.sites:
        if a_site in properties.keys():
            a_site.properties.update(properties[a_site])
        else:
            pass

    return copy_structure

In [10]:
def_struct = Structure.from_sites(defect_sites)

def_prop_struct = prop_in_struct(defect_properties, def_struct)
print(def_prop_struct)

Full Formula (X9 W4 Se1)
Reduced Formula: X9W4Se
abc   :  25.522526  25.522526  20.000000
angles:  90.000000  90.000000 120.000000
pbc   :       True       True       True
Sites (14)
  #  SP           a         b         c    an_change    new_an    original_an    substitution_defect    vacancy_defect
---  ----  --------  --------  --------  -----------  --------  -------------  ---------------------  ----------------
  0  W     0.041667  0.333333  0.185988           32        74             42                      1                 0
  1  W     0.166667  0.333333  0.185988           32        74             42                      1                 0
  2  W     0.166667  0.458333  0.185988           32        74             42                      1                 0
  3  X0+   0.417     0.958     0.186             -42         0             42                      0                 1
  4  X0+   0.542     0.958     0.186             -42         0             42                      0   

In [11]:
def get_nodes(structure):
    mdnn = MinimumDistanceNN()
    nodes = []
    for i, site in enumerate(structure.sites):
        the_cn = mdnn.get_cn(structure, i)
        node_features = [i, the_cn - 1, site.properties["original_an"], site.properties["new_an"],
                         site.properties["an_change"], site.properties["vacancy_defect"],
                         site.properties["substitution_defect"]]
        nodes.append(node_features)

    return nodes

print(get_nodes(def_prop_struct))

[[0, 1, 42, 74, 32, 0.0, 1.0], [1, 1, 42, 74, 32, 0.0, 1.0], [2, 1, 42, 74, 32, 0.0, 1.0], [3, 0, 42, 0, -42, 1.0, 0.0], [4, 0, 42, 0, -42, 1.0, 0.0], [5, 0, 42, 0, -42, 1.0, 0.0], [6, 0, 42, 74, 32, 0.0, 1.0], [7, 0, 42, 0, -42, 1.0, 0.0], [8, 0, 16, 34, 18, 0.0, 1.0], [9, 0, 16, 0, -16, 1.0, 0.0], [10, 1, 16, 0, -16, 1.0, 0.0], [11, 0, 16, 0, -16, 1.0, 0.0], [12, 0, 16, 0, -16, 1.0, 0.0], [13, 0, 16, 0, -16, 1.0, 0.0]]


In [ ]:
def get_edges(structure):
    sites_list = structure.sites

    edges = []
    edge_features = []

    home_site = []
    away_site = []

    for i, site_i in enumerate(sites_list):
        props = []
        for j, site_j  in enumerate(sites_list):
            if i != j:
                home_site.append(i)
                away_site.append(j)
            
                dist = site_i.distance(site_j)
                props.append(dist)
            edge_features.append(props)

            # if neighbors["site_index"] == j:
            #     props.append(neighbors["weight"])
            #     props.append(neighbors)

        

    edges.append(home_site)
    edges.append(away_site)
    # edge_features.append(props)
    return edges, edge_features

asd, df  = get_edges(def_prop_struct)



In [ ]:
list2 = []
# list1 = [1,2,3,4]
list1 = [4]
if max(list1) == 4:
    its_index = list1.index(4)
    list1[its_index] = 0
    list2.append(max(list1))
print(list2)

In [ ]:
def get_properties(new_defective_struct, all_defect_sites):
    list_of_sites = new_defective_struct.sites

    # Define an instance responsible for getting the nearest neighbors
    cnn = MinimumDistanceNN(cutoff=3.0)

    # dictionary of properties
    site_dict = {} # Of each site
    new_dict = {}  # Of all sites

    # Lets get the properties for each defect site
    for i in list_of_sites:
        the_index = list_of_sites.index(i)

        # Get the nearest neighbors of each defect site
        neighbors = cnn.get_nn_info(new_defective_struct, the_index)

        # Get the coordination numbers
        coord_numbers = len(neighbors)

        # Get the vital attributes:
        # neighbors
        the_neighbors = []
        # site index of neighbor
        neighbors_site_index = []
        # neighbors image
        neighbors_image = []
        # distance to neighbors
        bl_to_neighbors = []
        # weights to neighbors
        weights_to_neighbors = []
        # atomic numbers of neighbors
        elements_an = []

        for neighbor_info in neighbors:
            the_neighbors.append(neighbor_info['site'])
            neighbors_site_index.append(neighbor_info['site_index'])
            neighbors_image.append(neighbor_info['image'])
            bond_length = neighbor_info['site'].distance(i)
            bl_to_neighbors.append(bond_length)
            weights_to_neighbors.append(neighbor_info['weight'])
            # Handle the atomic numbers
            the_an = neighbor_info["site"].specie.Z
            if the_an > 0 and the_an < 119:
                elements_an.append(the_an)
            else:
                the_an = 0
                elements_an.append(the_an)
                
        # Get distance to other defect sites
        dist_to_other_defects = []
        for n in all_defect_sites:
            dist = i.distance(n)
            dist_to_other_defects.append(dist)

        # Collect these properties as dictionaries
        site_dict[i] = {"nearest_neighbors": the_neighbors,
                        "neighbors_atomic_numbers": elements_an,
                        "nearest_neighbors_index": neighbors_site_index,
                        "neighbors_image": neighbors_image,
                        "neighbors_weight": weights_to_neighbors,
                        "distance_to_neighbors": bl_to_neighbors,
                        "coordination_number": coord_numbers,
                        "distance_to_defects": dist_to_other_defects}
        
        new_dict.update(site_dict)
    
    return site_dict


In [ ]:
sites_properties = get_properties(new_defective_struct, all_defects)

for key1 in sites_properties.keys():
    if key1 in defect_an_property.keys():
        sites_properties[key1].update(defect_an_property[key1])
    else:
        pass

for site_struct in new_defective_struct.sites:
    if site_struct in sites_properties.keys():
        site_struct.properties.update(sites_properties[site_struct])
    else:
        pass

print(new_defective_struct)